In [1]:
from gurobipy import *
import random
from random import randint
#import numpy as np, numpy.random

# Optimization
# 1. Variables & Constraint

#### Let $B_T$ to be the total budget
#### Let $H = O \uplus V \uplus R= \{1 \dots n\}$ to be the set represents the housing stock of the city
#### Let $O \subseteq H$ to be the set represents the houses occupied by owner
#### Let $R \subseteq H$ to be the set represents the houses occupied by renter
#### Let $V \subseteq H$ to be the set represents the houses are vacant
#### Let $E$ to be the collection of all neighbors
#### Let $C = \{ (i,j) \; | \; i\in O\uplus R , j \in V \}$
###### Let $I_{V}(i)$ to be an indicator function represents that if the houses are vacant  
###### Let $I_{O}(i)$ to be an indicator function represents that if the houses are occupied by owner  
###### Let $I_{R}(i)$ to be an indicator function represents that if the houses are occupied by renter
###### Let $I_{S_2}(i)$ to be an indicator function represents that if the houses are 2-Stroy
###### Let $I_{S_3}(i)$ to be an indicator function represents that if the houses are 3-Story 

## - cost variables

###### Let $c_i$ to be the cost to demplish house i
>$$ c_i = 13000\times I_{S_2}(i) + 22000 \times I_{S_3}(i) + 85000 \times I_{R}(i) + 170000 \times I_{O}(i) $$

###### Let $w_{ij}$ to be the average price building the wall between houses ij, and $w_i$ and $w_j$ to be the difference between the price to build the wall for building $i/j$ minus $w_{ij}$ for all $(i,j) \in E$
>$$ w_{ij} = {14000\times I_{S_2}(i) + 25000\times I_{S_3}(i) + 14000\times I_{S_2}(j) + 25000\times I_{S_3}(j)\over 2 } $$
>$$ \mbox{ which won't be affacted by the value of} x_i \mbox{ and } x_j $$

>$$ w_i = (14000\times I_{S_2}(i) + 25000\times I_{S_3}(i)) - w_{ij} $$
>$$ w_j = (14000\times I_{S_2}(j) + 25000\times I_{S_3}(j)) - w_{ij} $$

>$\therefore$ The cost for building the wall will be
> $$ w_{ij}z_{ij} + w_i(1-x_i) + w_j(1-x_j) $$
>> Note that when houses ij have same height, $w_i$ and $w_j$ will be 0 

>> and when $z_{ij} = 0$, $w_i(1-x_i) + w_j(1-x_j) = 0$.

#### Let $p_{ij}$ to be th cost reduction building i,j demolish together for all $(i,j) \in E$
> Let's set $p_{ij}$ is same for neighbors


## - Optimization Variables

###### Let $x_i$  to be a binary variable  $\forall i \in H$, which if $x_i$ = 1 then house i would be demolished
###### Let $z_{ij} = x_i\oplus x_j \;\; \forall (ij) \in E$
$$ \begin{array}{rll}
    \text{s.t.} & x_i - x_j - z_{ij} \leq 0 \\
                & x_j - x_i - z_{ij} \leq 0 \\
                & x_i + x_j + z_{ij} \leq 2 \\
                &-x_i - x_j + z_{ij} \leq 0 \\ 
                & z_{ij} \in \{0,1\}
    \end{array}
$$
###### Let $y_{ij} = x_i\times x_j \;\; \forall (ij) \in E$
$$ \begin{array}{rll}
    \text{s.t.} & x_i + x_j - y_{ij} \leq 1 \\
                & -x_i + y_{ij} \leq 0 \\
                & -x_j + y_{ij} \leq 0 \\
                & y_{ij} = \{0,1\}
    \end{array}
$$



## - Total Cost Constraint

$$  
    \sum_{i} c_ix_i +  \sum_{(ij) \in E}w_{ij}(x_i \oplus x_j) + \sum_{(ij) \in E} w_i(1-x_i) + \sum_{(ij) \in E} w_j(1-x_j)  -\sum_{(ij) \in E}p_{ij}x_ix_j  \\ = \sum_{i} c_ix_i +  \sum_{(ij) \in E}w_{ij}z_{ij} + \sum_{(ij) \in E} w_i(1-x_i) + \sum_{(ij) \in E} w_j(1-x_j) -\sum_{(ij) \in E}p_{ij}y_{ij} \leq B_T
$$

- update version (01-10-18)

$$  
    \sum_{i} c_ix_i + \sum_{(ij) \in E} w_i(1-x_i)x_j + \sum_{(ij) \in E} w_j(1-x_j)x_i -\sum_{(ij) \in E}p_{ij}y_{ij} 
    \\ = \sum_{i} c_ix_i + \sum_{(ij) \in E} w_ix_j + \sum_{(ij) \in E} w_jx_i -\sum_{(ij) \in E}(w_i+w_j+p_{ij})y_{ij}  \leq B_T
$$

> - We only need $y_{ij}$
> - Note that $w_i$ and $w_j$ here is the cost for each building (not what we defined before)





# 2. Objective Function

## - Tearing down as many vacant houses as possible (Original)

> $$ \max \sum_{i\in V} x_i $$

## - Weight Function (New)

#### Let $h_{i,j} \in R \geq 0$ to be the function for all $i \in O\uplus R$ and $j \in V$ as measurement

> Note that $h_{i,j}$ is a function to measure the cumulative happiness/ profit/ the probibility to be safe for every pair $(i,j)$ where $i \in O\uplus R$ , $j \in V$ in the database

--------------


> ##### $h_{i,j}$ can be ... 

> - Cumulative risk for each house in $ O\uplus R$ : $\sum_{j\in V} h_{i,j}$

> - propotional to the ${1\over d^c}$

> - Indicator function 



# 3. ILP Model (01-10-18)

## - Weight Function ILP

$$ \begin{array}{rll}
    \text{min} & \sum_{i,j\in C} h_{i,j}(1-x_i)(1-x_j) \\
    =\text{min} & \sum_{i,j\in C} h_{i,j}(1-x_i -x_j-y_{ij}) \\
    \\[1pt] 
    \text{s.t.} &  \sum_{i} c_ix_i + \sum_{(ij) \in E} w_ix_j + \sum_{(ij) \in E} w_jx_i -\sum_{(ij) \in E}(w_i+w_j+p_{ij})y_{ij}  \leq B_T \\
    \\
    & \forall (i,j) \in E \uplus C\\
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E \uplus C\\
    \end{array}
$$

###### $\hspace{7cm}$ ~~(Plus the constraint for the delta method)~~


> - To save space, we can set all (i, j) pair in E to be in same format as C if possible. ( Depends on how we construct C )

- #### ~~Delta Method for Wight Function ILP (old slide)~~

> - New Variables

> Let $d_e$ to be the max distance that the vacant house will have affect the safety of the occupied house 

> Let $\delta_{ij} = (1-x_i)(1-x_j) \;\; \forall i \in O\uplus R \;,\;\forall j \in V$
$$ \begin{array}{rll}
    \text{s.t.} & -\delta_{ij} - x_i - x_j  \leq -1 \\
                & x_i + \delta_{ij} \leq 1 \\
                & x_j + \delta_{ij} \leq 1 \\
                & \delta_{ij} = \{0,1\}
    \end{array}
$$
> - Then the number of variables $$= |H| + 2|E| + |O\uplus R|\times|V| $$




> $\delta_{ij} $ will affect the result iff both houses i,j are not demolished.
$$ C = \{ (i,j) \;\;|\;\; i \in O \uplus R \;\;\text{ and }\;\; j \in V \} \\ $$
$$ \begin{array}{rll}
    & \forall\; (i,j)\in C \\
    & \;\;\;\;\;x_i + \delta_{ij} \leq 1 \\
    & \;\;\;\;\;x_j + \delta_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i - x_j - \delta_{ij} \leq -1 \\
    & \delta_{ij} \in \lbrace 0,1 \rbrace \\[1pt]\\
    \end{array}
$$

## - Weight Function ILP Transformation (Big M method)

$$ \min\sum_{i}\sum_{j} h_{i,j}(1-x_i)(1-x_j) $$
$$ = \max -\sum_{i}\sum_{j} h_{i,j}(1-x_i)(1-x_j) = \max \sum_{i}\sum_{j} h_{i,j}(1-x_i)(x_j-1) $$ 

> Therefore, when $x_i \neq 0 $

> We get

> $$ \min \sum_j h_{i,j} (1-x_j) = \max \sum_j h_{i,j} (x_j-1) $$

> then we can set $t_i$ to be

> $$ t_i \leq \sum_j h_{i,j}(x_j-1) \;,\; \mbox{ for all } j \in V$$

> and $$ t_i \leq M(1-x_i) $$

However, by the way we set up the firt inequality to be negative, no matter what $x_i$ is, the second inequality will always satisfy the first inequality. Thus, we should let the first bound to be sufficient large if the $x_i = 0$.

> $$ \therefore \;\;\forall \; i \in O\uplus R \;\;\; ,\;\; t_i \leq \sum_{j\in V} h_{i,j}(x_j-1)  + Mx_i $$


Thus when $x_i = 0$, $\max t_i = 0$ which means there is no effect for occupied house i. The objective optimal value will be the negative optimal value of the original problem

  $$\begin{array}{rll}
    & \forall \;i \in O \uplus R \\
    & t_{i} \leq \sum_{j\in V}h_{i,j}(x_j-1) + M(x_i) \\ 
    & t_{i} \in \{-\infty,0\} \\[1pt]\\
    & \text{max} \;\;\; \sum_{i \in O\uplus R} \; t_{i} \\
    \end{array}$$

Thus, The new ILP model will be

$$ \begin{array}{rll}
    \text{min} & \sum_{i,j\in C} h_{i,j}(1-x_i)(1-x_j) \\
    =\text{min} & \sum_{i,j\in C} h_{i,j}(1-x_i -x_j-y_{ij}) \\
    \\[1pt] 
    \text{s.t.} &  \sum_{i} c_ix_i + \sum_{(ij) \in E} w_ix_j + \sum_{(ij) \in E} w_jx_i -\sum_{(ij) \in E}(w_i+w_j+p_{ij})y_{ij}  \leq B_T \\
    \\
    & \forall (i,j) \in E \\
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    & \forall \;i \in O \uplus R \\
    & \;\;\;\;\; t_{i} \leq \sum_{j\in V}h_{i,j}(x_j-1) + M(x_i) \\
    \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E \\
    & t_i \; \in \lbrace -\infty,0 \rbrace \;\; \forall  i \in O\uplus R \\
    \end{array}
$$


### > Complexity Analysis after using Big M 

- num of ILP variables

> - Before: $$= |O\uplus R \uplus V| + |E| \uplus |O\uplus R|\times|V| \leq |H| + |E| + |C| $$

> - Big M Method: $$= |H| + |E| + |O\uplus R|$$

- num of constraint

> - Before: $$ \leq  1 + 3|E| + 3|C|$$

> - Big M Method: $$= 1 + 3|E| +  |O\uplus R|$$




# 4. Example 
$$ \begin{array}{rll} & x_1 \hspace{10cm} o_4 - x_2 - o_5\\ 
    & | \hspace{10cm}\;\;\;\;\;\;\;\;\;\; | \\
    & o_3 \hspace{10cm}\;\;\;\;\;\;\; o_6 
\end{array} $$

### Grid Map

In [2]:
from IPython.display import HTML as html_print
def PrintResult(m,x):
    border = "-"
    for i in xrange(len(m[0])):
        border += "-"
    border = 2*border 
    print border
    
    for i in xrange(len(m)):
        temp = "|"
        for j in xrange(len(m[i])):
            if m[i][j][0] != 0:
                if x[m[i][j]].X == 1.0 or (abs(x[m[i][j]].X - 1.0) <= 0.0000001):
                    temp += "\x1b[41m" + str(m[i][j][0]) + "\x1b[0m "  #ANSI escape code
                else:
                    temp += str(m[i][j][0]) + " "
            else:
                temp += "  "
                                
            
        print temp + "|"
    print border

In [3]:
def GetGridMap(size=15,example = None):
    
    Size = size
    CRoads = random.sample(xrange(1,Size-2), randint(2,Size/2))
    RRoads = random.sample(xrange(1,Size-2), randint(2,Size/2))
    Map = [[[0,(i,j),None] if (i == 0 or j==0 
                    or i == Size-1 or j == Size-1 
                    or i in RRoads or j in CRoads) else [1,(i,j),None] 
            for i in xrange(Size)] 
                for j in xrange(Size)]
    
    return Map

In [4]:
def HouseGenerator(m,example = None):
    foo = ["r","o","v"]
    if example == None:
        for i in xrange(len(m)):
            for j in xrange(len(m[i])):
                if m[i][j][0] != 0:
                    ranfoo = random.randint(0,2)
                    m[i][j][0] = foo[ranfoo]
                    m[i][j][2] = random.randint(2,3)
    elif example == 1:
        for i in xrange(len(m)):
            for j in xrange(len(m[i])):
                m[i][j][0] = 0
        houses = [("r",4,1),("v",6,1),("r",5,7),("v",5,9),("r",3,9),("r",7,9)]
        
        for h,i,j in houses:
            m[i][j][0] = h
            m[i][j][2] = 2
    elif example == 2:
        for i in xrange(len(m)):
            for j in xrange(len(m[i])):
                m[i][j][0] = 0
        houses = [("v",2,3),("v",2,4),("v",2,5),("v",2,6),("v",2,7),
                  ("v",3,3),("v",4,3),("v",5,3),("v",6,3),
                  ("v",6,4),("v",6,5),("v",6,6),("v",6,7),
                  ("v",5,7),("v",4,7),("v",3,7),("o",4,5)
                  ]
        
        for h,i,j in houses:
            m[i][j][0] = h
            m[i][j][2] = 2
    

def GetTupleMap(m):
    for i in xrange(len(m)):
        for j in xrange(len(m[i])):
            m[i][j] = tuple(m[i][j])
            
def PrintMap(m):
    border = "-"
    for i in xrange(len(m[0])):
        border += "-"
    border = 2*border 
    print border
    for row in m:
        temp = "|"
        for item in row:
            temp += (str(item[0]) + " " if item[0] != 0 else "  ")
        print temp + "|"
    print border
    
def GetEdgeSet(m):
    E = []
    for i in xrange(len(m)):
        for j in xrange(len(m[i])):
            if m[i][j][0] != 0 :
                if i+1 < len(m) and m[i+1][j][0] != 0:
                    E.append((m[i][j],m[i+1][j]))
                if j+1 < len(m[i]) and m[i][j+1][0] != 0:
                    E.append((m[i][j],m[i][j+1]))
    return E

In [5]:
def GetHouseSet(m):
    H = []
    for i in xrange(len(m)):
        for j in xrange(len(m[i])):
            if m[i][j][0] != 0:
                H.append(m[i][j])
    return H
def GetOwnerSet(H):
    O = []
    for item in H:
        if item[0] == 'o':
            O.append(item)
    return O
def GetRenterSet(H):
    R = []
    for item in H:
        if item[0] == 'r':
            R.append(item)
    return R
def GetVacantSet(H):
    V = []
    for item in H:
        if item[0] == 'v':
            V.append(item)
    return V
def GetCompareHousesSet(O,R,V):
    C = []
    for owner in O:
        for vacant in V:
            C.append((owner,vacant))
    for renter in R:
        for vacant in V:
            C.append((renter,vacant))
    return C

In [6]:
def distance(x1,x2):
    dis = abs(x2[1][0]-x1[1][0]) + abs(x2[1][1]-x1[1][1])
    return dis 
#print distance(Map[1][1],Map[6][8])

In [7]:
def affect(x1,x2):
    dis = distance(x1,x2)
    return 1.0/dis if dis <= 7 else 0
#print affect(Map[1][1],Map[1][2])

In [117]:
Map = GetGridMap(50)
HouseGenerator(Map, example = None)
GetTupleMap(Map)
PrintMap(Map)
Edge = GetEdgeSet(Map)
Houses = GetHouseSet(Map)
Owners = GetOwnerSet(Houses)
Renters = GetRenterSet(Houses)
Vacants = GetVacantSet(Houses)
CompareHouses = GetCompareHousesSet(Owners,Renters,Vacants)
# matplotlib,
Occupied = Owners + Renters
print "Total Houses:", len(Houses)

------------------------------------------------------------------------------------------------------
|                                                                                                    |
|        v   r o v     v o     v   v o v       o     r o r   v v       v   r r       o   v   o v r   |
|        v   r o r     r r     v   o o v       v     o r r   v v       v   o v       o   r   o v r   |
|                                                                                                    |
|        v   v o o     o o     o   v r o       r     o r o   r o       v   r r       o   v   r o o   |
|        o   r o o     r o     v   v o v       r     o o o   o r       o   v o       v   v   r o r   |
|                                                                                                    |
|        o   v v r     o v     r   r v v       v     v r v   v r       o   v v       o   r   r o o   |
|        v   r r o     r r     v   o v o       v     r v v   o v       r 

In [118]:
#print "Variables:" , len(CompareHouses) + len(Houses) + 2*len(Edge)

### Approximate cost 

In [119]:
Budget = 100000
demolish_2_story, demolish_3_story = 13000, 22000
r_relocate, o_relocate = 85000, 170000
wall_2_story, wall_3_story = 14000, 25000
cost_reduction = 2000

In [120]:
# cost
Cost = [ (13000 if item[2]==2 else 0) +
      (22000 if item[2]==3 else 0) +
      (85000 if item[0]== 'r' else 0) + 
      (85000 if item[0]== 'o' else 0) #170000
        for item in Houses]

# Wall cost
Wallij = [ (7000 if item[0][2] == 2 else 0) +
      (7000 if item[1][2] == 2 else 0) +
      (12500 if item[0][2] == 3 else 0) + 
      (12500 if item[1][2] == 3 else 0)
     for item in Edge]

Walli = [ (14000 if Edge[i][0][2] == 2 else 0) +
      (25000 if Edge[i][0][2] == 3 else 0) - 
       Wallij[i]
     for i in xrange(len(Edge))]

Wallj = [ (14000 if Edge[i][1][2] == 2 else 0) +
      (25000 if Edge[i][1][2] == 3 else 0) - 
       Wallij[i] 
         for i in xrange(len(Edge))]

#print Wij
#print Wi,Wj

# Rdduction
Benefit = [ cost_reduction
     for i in xrange(len(Edge))]

# 01-10-18
wi = [ (14000 if Edge[i][0][2] == 2 else 0) +
      (25000 if Edge[i][0][2] == 3 else 0)
     for i in xrange(len(Edge))]

wj = [ (14000 if Edge[i][1][2] == 2 else 0) +
      (25000 if Edge[i][1][2] == 3 else 0)
         for i in xrange(len(Edge))]

In [121]:
print "Cost:" , Cost[:5]
print "Wallij:" , Wallij[:5]
print "Walli:", Walli[:5]
print "Wallj:", Wallj[:5]
print "Wall for i:" , [Wallij[i]+Walli[i] for i in xrange(len(Wallij))][:5]
print "Wall for j:" , [Wallij[i]+Wallj[i] for i in xrange(len(Wallij))][:5]
print "cost reduction:" , Benefit[:5]

Cost: [13000, 98000, 107000, 13000, 22000]
Wallij: [19500, 19500, 19500, 19500, 19500]
Walli: [-5500, -5500, -5500, 5500, 5500]
Wallj: [5500, 5500, 5500, -5500, -5500]
Wall for i: [14000, 14000, 14000, 25000, 25000]
Wall for j: [25000, 25000, 25000, 14000, 14000]
cost reduction: [2000, 2000, 2000, 2000, 2000]


In [122]:
# 01-10-18 update
print  wi == [Wallij[i]+Walli[i] for i in xrange(len(Wallij))]
print  wj == [Wallij[i]+Wallj[i] for i in xrange(len(Wallij))]

True
True


### Weight Function ILP Transformation (Big M method)

In [123]:
test0 = Model() # empty

x = test0.addVars(Houses,vtype = GRB.BINARY,name = "x")
#z = test0.addVars(Edge,vtype = GRB.BINARY,name = "z")
y = test0.addVars(Edge,vtype = GRB.BINARY,name = "y")
bigM = test0.addVars(Occupied,vtype = GRB.CONTINUOUS,name = "bigM",lb = -GRB.INFINITY, ub = 0.0)

In [124]:
Budget_Constraint = test0.addConstr((quicksum(Cost[i]*x[Houses[i]] for i in xrange(len(Houses))) -
                     quicksum((Benefit[i]+wi[i]+wj[i])*y[Edge[i]] for i in xrange(len(Edge))) +
                     quicksum(wi[i]*x[Edge[i][1]] for i in xrange(len(Edge))) +
                     quicksum(wj[i]*x[Edge[i][0]] for i in xrange(len(Edge))) 
                     <= 
                     Budget 
                    )
                     , name = "Budget_Constraint")
"""
XOR1 = test0.addConstrs((x[Edge[i][0]] - x[Edge[i][1]] - z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR1")
XOR2 = test0.addConstrs((x[Edge[i][1]] - x[Edge[i][0]] - z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR2")
XOR3 = test0.addConstrs((x[Edge[i][1]] + x[Edge[i][0]] + z[Edge[i]] <= 2 
                      for i in xrange(len(Edge))
                     ),name = "XOR3")
XOR4 = test0.addConstrs((-x[Edge[i][1]] - x[Edge[i][0]] + z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR4")
"""

CD1 = test0.addConstrs((x[Edge[i][1]] + x[Edge[i][0]] - y[Edge[i]] <= 1 
                      for i in xrange(len(Edge))
                     ),name = "CD1")

CD2 = test0.addConstrs((-x[Edge[i][1]]  + y[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "CD2")

CD3 = test0.addConstrs((-x[Edge[i][0]]  + y[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "CD3")

In [125]:
for h in Occupied:
    total = sum( [affect(h,v) for v in Vacants])
    test0.addConstr(( bigM[h] <= quicksum(affect(h,v)*(x[v]-1) for v in Vacants) + 1.*total*x[h] ) , name = "for each occupied") 
    #test0.addConstr(( bigM[h] <= 100000000000*(1-x[h]) ))
                     #+ quicksum(affect(h,v) for v in Vacants)*x[h]), name = str(h))
            
            
test0.setObjective( quicksum(bigM[i] for i in Occupied), GRB.MAXIMIZE)

In [126]:
test0.optimize()

Optimize a model with 3945 rows, 2628 columns and 19668 nonzeros
Variable types: 602 continuous, 2026 integer (2026 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective -2384.09
Presolve time: 0.08s
Presolved: 3945 rows, 2628 columns, 20045 nonzeros
Variable types: 580 continuous, 2048 integer (2026 binary)

Root relaxation: objective -2.345694e+03, 2785 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2345.6939    0    6 -2384.0857 -2345.6939  1.61%     -    0s
H    0     0                    -2348.683333 -2345.6939  0.13%     -    0s
     0     0 -2346.3532    0    4 -2348.6833 -2346.3532  0.10%     -    0s

Cutting planes:
  Gomory: 1

Explored 0 nodes (2786 simplex iterations) in 0.29 seconds
T

In [127]:
#test0.printAttr('x')

- #### Test Objective Optimal Value

In [128]:
checkobject = 0
che = 0
for h in Occupied:
    checkobject += sum(affect(h,v)*(1-x[v].x) for v in Vacants)*(1-x[h].x)
    che += x[h].x
print checkobject

2348.68333333


In [129]:
sum(bigM[i].x for i in Occupied)

-2348.683333333331

In [130]:
test0.ObjVal

-2348.683333333331

In [131]:
a = 0
for j in Occupied:
    for i in Vacants:
        a += affect(j,i)*(1-x[i].x)*(1-x[j].x)
a

2348.68333333335

#### Result

In [132]:
PrintResult(Map,x)
print sum(bigM[i].x for i in Occupied)

------------------------------------------------------------------------------------------------------
|                                                                                                    |
|        v   r o v     v o     v   v o v       o     r o r   v v       v   r r       o   v   o v r   |
|        v   r o r     r r     v   o o v       v     o r r   v v       v   o v       o   r   o v r   |
|                                                                                                    |
|        v   v o o     o o     o   v r o       r     o r o   r o       v   r r       o   v   r o o   |
|        o   r o o     r o     v   v o v       r     o o o   o r       o   v o       v   v   r o r   |
|                                                                                                    |
|        o   v v r     o v     r   r v v       v     v r v   v r       o   v v       o   r   r o o   |
|        v   r r o     r r     v   o v o       v     r v v   o v       r 

### Weight Function ILP  (2017 summer one)

In [133]:
test = Model() # empty

x = test.addVars(Houses,vtype = GRB.BINARY,name = "x")
z = test.addVars(Edge,vtype = GRB.BINARY,name = "z")
y = test.addVars(Edge,vtype = GRB.BINARY,name = "y")
delta = test.addVars(CompareHouses,vtype = GRB.BINARY,name = "delta")

In [134]:
test.update()


Budget_Constraint = test.addConstr((quicksum(Cost[i]*x[Houses[i]] for i in xrange(len(Houses))) +
                     quicksum(Wallij[i]*z[Edge[i]] for i in xrange(len(Edge))) -
                     quicksum(Benefit[i]*y[Edge[i]] for i in xrange(len(Edge))) -
                     quicksum(Walli[i]*x[Edge[i][0]] for i in xrange(len(Edge))) -
                     quicksum(Wallj[i]*x[Edge[i][1]] for i in xrange(len(Edge))) 
                     <= 
                     Budget -
                     quicksum(Walli) - quicksum(Wallj)
                    )
                     , name = "Budget_Constraint")

XOR1 = test.addConstrs((x[Edge[i][0]] - x[Edge[i][1]] - z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR1")
XOR2 = test.addConstrs((x[Edge[i][1]] - x[Edge[i][0]] - z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR2")
XOR3 = test.addConstrs((x[Edge[i][1]] + x[Edge[i][0]] + z[Edge[i]] <= 2 
                      for i in xrange(len(Edge))
                     ),name = "XOR3")
XOR4 = test.addConstrs((-x[Edge[i][1]] - x[Edge[i][0]] + z[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "XOR4")

CD1 = test.addConstrs((x[Edge[i][1]] + x[Edge[i][0]] - y[Edge[i]] <= 1 
                      for i in xrange(len(Edge))
                     ),name = "CD1")

CD2 = test.addConstrs((-x[Edge[i][1]]  + y[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "CD2")

CD3 = test.addConstrs((-x[Edge[i][0]]  + y[Edge[i]] <= 0 
                      for i in xrange(len(Edge))
                     ),name = "CD3")

detlatConstraint1 = test.addConstrs((x[CompareHouses[i][0]]  + delta[CompareHouses[i]] <= 1 
                      for i in xrange(len(CompareHouses))
                     ),name = "detlatConstraint1")
detlatConstraint1 = test.addConstrs((x[CompareHouses[i][1]]  + delta[CompareHouses[i]] <= 1 
                      for i in xrange(len(CompareHouses))
                     ),name = "detlatConstraint1")
detlatConstraint1 = test.addConstrs(( -x[CompareHouses[i][1]]-x[CompareHouses[i][0]]  
                                     - delta[CompareHouses[i]] <= -1 
                      for i in xrange(len(CompareHouses))
                     ),name = "detlatConstraint1")


#c2 = m.addConstr(x + y >= 1)

In [135]:
test.setObjective( quicksum(affect(pare[0],pare[1])*delta[pare] for pare in CompareHouses) , GRB.MINIMIZE)
#affect(pare[0],pare[1])*delta[pare] for pare in CompareHouses

In [136]:
test.optimize()

Optimize a model with 567659 rows, 189760 columns and 1330646 nonzeros
Variable types: 0 continuous, 189760 integer (189760 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (presolve time = 10s) ...
Presolve time: 13.11s
Presolved: 567659 rows, 189760 columns, 1330646 nonzeros
Variable types: 0 continuous, 189760 integer (189760 binary)
Found heuristic solution: objective 2384.0857143

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.866200e+05   0.000000e+00     18s
   26120    8.9442481e+01   1.558400e+05   0.000000e+00     20s
  188892    2.3433792e+03   3.856792e+02   0.000000e+00     25s
  188975    2.3456939e+03   0.000000e+00   0.000000e+00     26s

Root relaxation: objective 2.345694e+03, 188975 i

In [137]:
#for v in test.getVars():
    #if v.X != 0:
    #print("%s %f" % (v.Varname, v.X))

#### Result

In [138]:
PrintResult(Map,x)
print test.ObjVal

------------------------------------------------------------------------------------------------------
|                                                                                                    |
|        v   r o v     v o     v   v o v       o     r o r   v v       v   r r       o   v   o v r   |
|        v   r o r     r r     v   o o v       v     o r r   v v       v   o v       o   r   o v r   |
|                                                                                                    |
|        v   v o o     o o     o   v r o       r     o r o   r o       v   r r       o   v   r o o   |
|        o   r o o     r o     v   v o v       r     o o o   o r       o   v o       v   v   r o r   |
|                                                                                                    |
|        o   v v r     o v     r   r v v       v     v r v   v r       o   v v       o   r   r o o   |
|        v   r r o     r r     v   o v o       v     r v v   o v       r 

### Compare

In [139]:
print test0.printQuality
print test.printQuality , "\n\n\n"
print "Running Time of Big M: " ,test0.Runtime
print "Running Time of 2017 Summer: " ,test.Runtime , "\n\n\n"

print test.ObjVal == checkobject
print test.ObjVal, checkobject
print -test0.ObjVal == test.ObjVal
print test.ObjVal, -test0.ObjVal

<bound method Model.printQuality of <gurobi.Model MIP instance Unnamed: 3945 constrs, 2628 vars, Parameter changes: LogFile=gurobi.log>>
<bound method Model.printQuality of <gurobi.Model MIP instance Unnamed: 567659 constrs, 189760 vars, Parameter changes: LogFile=gurobi.log>> 



Running Time of Big M:  0.298236131668
Running Time of 2017 Summer:  470.625692129 



False
2348.68333333 2348.68333333
False
2348.68333333 2348.68333333


In [140]:
temp = 0
che = 0
for h in Occupied:
    temp += sum(affect(h,v)*(1-x[v].x) for v in Vacants)*(1-x[h].x)
    che += x[h].x
print temp

2348.68333333


In [141]:
temp == checkobject

True

-----------------------------------------------------------------------------------------------------------------

# $\hspace{10cm}$  Above 01-10-2018

## Extra Note

- The Big M method will tend to demolish all houses in some case due to the affect of tolerance

- Maximize the minimum distance between occupied and vacant

> $$\begin{array}{rll}
    & \forall \;i \in O \uplus R \\
    & t_{i} \leq d_{i,j}(1-x_j) + M x_j \;\; \forall j \in V \\ 
    & t_{i} \leq M(1-x_i) \\
    & t_{i} \in \text{Real Number} \\[1pt]\\
    & \text{max} \;\;\; \sum_{i \in O\uplus R} \;t_{i} \\
    \end{array}$$

9/27
> $$\begin{array}{rll}
    & \forall \;i \in O \uplus R \\
    & t_{i} \leq d_{i,j}(1-x_j) + M x_j + M x_i\;\; \forall j \in V \\ 
    & t_{i} \in \text{Real Number} \\[1pt]\\
    & \text{max} \;\;\; \sum_{i \in O\uplus R} \;t_{i} \\
    \end{array}$$